In [8]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "Have been having a hard time lately with sleep. Need prayers for peace and rest"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1) negative 0.7597
2) neutral 0.2109
3) positive 0.0294


In [11]:
from transformers import pipeline
from pprint import pprint

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

sentences = [text]

model_outputs = classifier(sentences)
pprint(model_outputs[0])
# produces a list of dicts for each of the labels


[{'label': 'desire', 'score': 0.397585391998291},
 {'label': 'caring', 'score': 0.3546721637248993},
 {'label': 'sadness', 'score': 0.29200223088264465},
 {'label': 'neutral', 'score': 0.044636670500040054},
 {'label': 'optimism', 'score': 0.037261370569467545},
 {'label': 'disappointment', 'score': 0.037139419466257095},
 {'label': 'remorse', 'score': 0.0341009646654129},
 {'label': 'approval', 'score': 0.01934124529361725},
 {'label': 'nervousness', 'score': 0.01838996633887291},
 {'label': 'love', 'score': 0.014858935959637165},
 {'label': 'annoyance', 'score': 0.014482901431620121},
 {'label': 'curiosity', 'score': 0.012793241068720818},
 {'label': 'grief', 'score': 0.009511505253612995},
 {'label': 'joy', 'score': 0.009283608756959438},
 {'label': 'disapproval', 'score': 0.008858146145939827},
 {'label': 'realization', 'score': 0.007423378527164459},
 {'label': 'relief', 'score': 0.007150622084736824},
 {'label': 'gratitude', 'score': 0.007124544586986303},
 {'label': 'admiration'